<a href="https://colab.research.google.com/github/emh0841-code/Final-Team9/blob/main/%EC%8B%9C%EB%8C%80%EC%97%90_%EB%94%B0%EB%9D%BC_%ED%9D%A5%ED%96%89%ED%95%98%EB%8A%94_%EC%9E%A5%EB%A5%B4%EA%B0%80_%EB%8B%A4%EB%A5%BC%EA%B2%83%EC%9D%B4%EB%8B%A4_9/28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#한글 글씨 폰트 설치
%%capture
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm.fontManager.addfont('/usr/share/fonts/truetype/nanum/NanumGothic.ttf')
plt.rcParams['font.family'] = 'NanumGothic'

# 표에서 ('-') 마이너스 표시
plt.rcParams['axes.unicode_minus'] = False

# 필요한 라이브러리 임포트
import pandas as pd # 데이터 분석 라이브러리
import numpy as np
import matplotlib.pyplot as plt # 시각화 도구 라이브러리1
import seaborn as sns # 시각화 도구 라이브러리2

In [2]:
df = pd.read_csv("/content/movies_genres_ohe.csv", on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: '/content/movies_genres_ohe.csv'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, f_oneway, kruskal

df.columns = df.columns.str.strip()

# 날짜 → year
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year

# ===== 1) 1990~2025 구간 필터 =====
df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()

# ===== 2) 5년 단위 구간 만들기 =====
bins   = list(range(1990, 2030, 5))   # [1990,1995,...,2025]
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["period"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)

# ===== 3) 장르(0/1) 컬럼 추출 =====
cols = df.loc[:, "Action":"Western"].columns.tolist()

# ===== 4) 장르별 전체 편수 확인 =====
col_counts = df[cols].sum().sort_values(ascending=False)
print("[1990~2025] 장르별 전체 편수:\n", col_counts, "\n")

# ===== 5) 시대 × 장르별 y_result 평균 =====
period_genre = {}
for c in cols:
    period_genre[c] = df.loc[df[c] == 1].groupby("period")["y_result"].mean()

period_genre = pd.DataFrame(period_genre).sort_index()
print("5년 단위 × 장르별 평균 y_result (1990~2025):")
print(period_genre, "\n")

# ===== 6) 히트맵 시각화 =====
plt.figure(figsize=(14,8))
sns.heatmap(period_genre, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("5년 단위 × 장르별 평균 y_result (1990~2025)")
plt.xlabel("장르")
plt.ylabel("Period")
plt.show()

# ===== 7) 전체 기간 평균 기준 상위 5개 장르 =====
topN = 5
top_cols = period_genre.mean().sort_values(ascending=False).head(topN).index.tolist()
print(f"1990~2025 상위 {topN}개 장르:", top_cols, "\n")

# 라인그래프 (Top5 트렌드)
period_genre[top_cols].plot(marker="o", figsize=(14,6))
plt.title(f"5년 단위 평균 y_result 흐름 (1990~2025, 상위 {topN} 장르)")
plt.xlabel("Period")
plt.ylabel("평균 y_result")
plt.legend(title="장르")
plt.show()


In [ ]:
from scipy.stats import shapiro, levene, f_oneway, kruskal

# 1) 정규성 검정 (Shapiro-Wilk)
print("[정규성 검정: Shapiro-Wilk] (1990~2025, 전체 장르)")
normality_results = {}
for c in cols:  # cols = ['Action', ..., 'Western']
    vals = df.loc[df[c] == 1, "y_result"].dropna()
    if len(vals) >= 8:  # 샘플이 너무 적으면 제외
        stat, p = shapiro(vals)
        normality_results[c] = (stat, p, len(vals))
        print(f"{c:15s} → W={stat:.4f}, p={p:.4e}, n={len(vals)}")
    else:
        print(f"{c:15s} → 샘플 부족 (n={len(vals)})")

# 2) 등분산성 검정 (Levene test, 모든 장르 비교)
groups = [df.loc[df[c] == 1, "y_result"].dropna() for c in cols if len(df.loc[df[c] == 1, "y_result"].dropna()) > 1]
stat, p = levene(*groups)
print(f"\n[등분산성 검정: Levene]\nW={stat:.4f}, p={p:.4e}")

# 3) ANOVA & Kruskal-Wallis
if all(pv > 0.05 for _, pv, _ in normality_results.values()):
    # 전체 장르가 정규성을 만족한다면 ANOVA
    f_stat, p_val = f_oneway(*groups)
    print(f"\n[ANOVA 결과]\nF={f_stat:.4f}, p={p_val:.4e}")
else:
    # 정규성 깨지면 Kruskal-Wallis
    h_stat, p_val = kruskal(*groups)
    print(f"\n[Kruskal-Wallis 결과]\nH={h_stat:.4f}, p={p_val:.4e}")


1990~2025년 전체 장르 간 y_result 분포에는 매우 유의한 차이

최신기간기준 top5

In [ ]:

# 0) year & 5년 구간(period) 만들기
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()

bins   = list(range(1990, 2030, 5))                 # [1990,1995,...,2025]
labels = [f"{y}–{y+4}" for y in bins[:-1]]          # '1990–1994' 형식
df["period"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)

# 1) 장르 컬럼
cols = df.loc[:, "Action":"Western"].columns.tolist()

# 2) '2020–2024'에서 장르별 평균 y_result → Top5
latest_period = "2020–2024"
df_latest = df[df["period"] == latest_period].copy()

genre_mean_latest = {
    c: df_latest.loc[df_latest[c]==1, "y_result"].mean() for c in cols
}
genre_mean_latest = pd.Series(genre_mean_latest).dropna().sort_values(ascending=False)

topN = 5
top_cols_latest = genre_mean_latest.head(topN).index.tolist()
print(f"{latest_period} Top{topN} 장르:", top_cols_latest)

# 3) 전체(1990~2025)에서 이 Top5 장르의 5년 단위 평균 y_result 흐름
period_trend = {}
for c in top_cols_latest:
    period_trend[c] = df.loc[df[c]==1].groupby("period")["y_result"].mean()

period_trend = pd.DataFrame(period_trend).reindex(labels)  # 구간 순서 보장
print(f"\n{latest_period} Top{topN} 장르의 5년 단위 평균 y_result 흐름:")
print(period_trend)

# 4) 시각화
ax = period_trend.plot(marker="o", figsize=(12,6))
ax.set_title(f"{latest_period} Top{topN} 장르의 5년 단위 y_result 흐름")
ax.set_xlabel("5년 구간")
ax.set_ylabel("평균 y_result")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


전체기간 기준으로 top5

In [ ]:


# ===== 0) year & 5년 구간 만들기
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()

bins   = list(range(1990, 2030, 5))                 # [1990,1995,...,2025]
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["period"] = pd.cut(df["year"], bins=bins, labels=labels,
                      right=True, include_lowest=True)

# ===== 1) 장르 컬럼
cols = df.loc[:, "Action":"Western"].columns.tolist()

# ===== 2) 전체 구간에서 장르별 평균 y_result
genre_mean_all = {
    c: df.loc[df[c]==1, "y_result"].mean() for c in cols
}
genre_mean_all = pd.Series(genre_mean_all).dropna().sort_values(ascending=False)

topN = 5   # 상위 몇 개 장르를 볼지
top_cols_all = genre_mean_all.head(topN).index.tolist()
print(f"1990~2025 전체 평균 기준 Top{topN} 장르:", top_cols_all)

# ===== 3) 이 TopN 장르들의 5년 단위 평균 y_result 트렌드
period_trend = {}
for c in top_cols_all:
    period_trend[c] = df.loc[df[c]==1].groupby("period")["y_result"].mean()

period_trend = pd.DataFrame(period_trend).reindex(labels)  # 구간 순서 보장

print(f"\nTop{topN} 장르의 5년 단위 평균 y_result 트렌드:")
print(period_trend)

# ===== 4) 시각화
plt.figure(figsize=(12,6))
for c in top_cols_all:
    plt.plot(period_trend.index, period_trend[c], marker="o", label=c)

plt.title(f"Top{topN} 장르 (1990~2025 전체 평균 기준)의 5년 단위 y_result 흐름")
plt.xlabel("5년 구간")
plt.ylabel("평균 y_result")
plt.xticks(rotation=45, ha="right")
plt.legend(title="장르")
plt.tight_layout()
plt.show()


최신기간 기준

In [ ]:

# ===== 1) release_year & year_bin 생성 =====
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df = df[df["release_year"].notna()].copy()
df["release_year"] = df["release_year"].astype(int)

# 5년 단위 bin
df["year_bin"] = (df["release_year"] // 5) * 5

# 1990~2025년만 사용
df = df[df["year_bin"] >= 1990].copy()
df.loc[df["release_year"] >= 2025, "year_bin"] = 2025  # 2025 이후 데이터 포함

# ===== 2) 장르 컬럼 자동 추출 =====
cols = df.loc[:, "Action":"Western"].columns.tolist()

# ===== 3) 최신 구간 찾기 =====
latest_bin = df["year_bin"].max()

# 최신 구간 데이터만
df_latest = df[df["year_bin"] == latest_bin]

# 최신 구간에서 장르별 평균 y_result
genre_mean_latest = {
    c: df_latest.loc[df_latest[c] == 1, "y_result"].mean() for c in cols
}
genre_mean_latest = pd.Series(genre_mean_latest).dropna().sort_values(ascending=False)

# 최신 구간 Top5 장르
top5 = genre_mean_latest.head(5).index.tolist()
print(f" {latest_bin}년대 기준 Top5 장르:", top5)

# ===== 4) Top5 장르의 5년 단위 평균 y_result 추세 =====
trend_top5 = {c: df.loc[df[c] == 1].groupby("year_bin")["y_result"].mean() for c in top5}
trend_top5 = pd.DataFrame(trend_top5).sort_index()

# 각 구간별 비율(%) 변환 (합=100%)
trend_top5_pct = trend_top5.div(trend_top5.sum(axis=1).replace(0, np.nan), axis=0) * 100

# ===== 5) 시각화 (스택형 막대그래프, 100% 비율) =====
ax = trend_top5_pct.plot(kind="bar", stacked=True, figsize=(14,7), alpha=0.9)
ax.set_title(f"최신구간 기준 1990~{latest_bin} (5년 단위) Top5 장르 y_result 비율 (100%)")
ax.set_xlabel("Year Bin (5년 단위)")
ax.set_ylabel("비율 (%)")
ax.legend(title="장르", bbox_to_anchor=(1.02, 1), loc="upper left")

# 막대 내부에 % 라벨 (8% 이상만 표시)
for container in ax.containers:
    labels = [f"{h.get_height():.0f}%" if h.get_height() >= 8 else "" for h in container]
    ax.bar_label(container, labels=labels, label_type='center', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# ===== 0) 연도 & 5년 단위 구간(period) 만들기 =====
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()

bins   = list(range(1990, 2030, 5))            # [1990,1995,...,2025]
labels = [f"{y}–{y+4}" for y in bins[:-1]]     # '1990–1994' 형식
df["period"] = pd.cut(df["year"], bins=bins, labels=labels,
                      right=True, include_lowest=True)

# ===== 1) 전체 장르별 5년 단위 평균 y_result 계산 =====
period_trend_all = {}
for c in cols:  # cols = ['Action','Adventure',...,'Western']
    period_trend_all[c] = df.loc[df[c] == 1].groupby("period")["y_result"].mean()

period_trend_all = pd.DataFrame(period_trend_all).reindex(labels)  # 구간 순서 맞춤

print("\n전체 장르의 5년 단위 평균 y_result 흐름:")
print(period_trend_all)

# ===== 2) 시각화 =====
plt.figure(figsize=(14,7))
for c in period_trend_all.columns:
    plt.plot(period_trend_all.index, period_trend_all[c], marker="o", label=c, alpha=0.7)

plt.title("전체 장르의 5년 단위 y_result 트렌드 (1990~2025)")
plt.xlabel("5년 구간")
plt.ylabel("평균 y_result")
plt.xticks(rotation=45, ha="right")
plt.legend(title="장르", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()



In [ ]:
# 0) 준비: 연도/5년 주기
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df["year_bin"] = (df["release_year"] // 5) * 5
df = df[df["year_bin"] >= 1990].copy()
df.loc[df["release_year"] >= 2025, "year_bin"] = 2025  # 2025 포함

# 1) 전체 장르별 5년 주기 평균 y_result 계산
trend_all = {}
for c in cols:   # cols = ['Action','Adventure',...,'Western'] 장르 리스트
    trend_all[c] = df.loc[df[c]==1].groupby("year_bin")["y_result"].mean()

trend_all = pd.DataFrame(trend_all).sort_index()

# 2) 100% 정규화
trend_all_pct = trend_all.div(trend_all.sum(axis=1), axis=0) * 100

# 3) 초기↔최신 변화(퍼센트p) 계산
first_bin = trend_all_pct.index.min()
latest_bin = trend_all_pct.index.max()
change_pp_all = (trend_all_pct.loc[latest_bin] - trend_all_pct.loc[first_bin]).sort_values()

# 4) 시각화: 왼쪽 = 100% 누적 막대(전체 장르), 오른쪽 = 변화(퍼센트p)
fig = plt.figure(figsize=(18,9))
gs = fig.add_gridspec(1, 2, width_ratios=[2.5, 1])

# (a) 100% 누적 막대
ax1 = fig.add_subplot(gs[0,0])
trend_all_pct.plot(kind="bar", stacked=True, ax=ax1, alpha=0.9)
ax1.set_title("1990~2025 (5년 주기) 전체 장르의 y_result 비율 (100%)")
ax1.set_xlabel("Year Bin (5년 단위)")
ax1.set_ylabel("비율 (%)")
ax1.legend(title="장르", bbox_to_anchor=(1.02, 1), loc="upper left")

# 막대 라벨(퍼센트 표시, 작은 값 생략)
for container in ax1.containers:
    labels = []
    for h in container:
        val = h.get_height()
        labels.append(f"{val:.0f}%" if val >= 8 else "")
    ax1.bar_label(container, labels=labels, label_type='center', fontsize=8)




In [ ]:


# 2) 연도 추출
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[df["year"] >= 1980].copy()

# 3) 5년 단위 구간 생성
bins = list(range(1980, 2025, 5))   # [1980, 1985, ..., 2020]
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["five_year_bin"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)

# 4) 장르 컬럼 정의 (0/1 더미)
genre_cols = [
    'Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family',
    'Fantasy','History','Horror','Music','Mystery','Romance','Science Fiction',
    'TV Movie','Thriller','War','Western'
]
genre_cols = [c for c in genre_cols if c in df.columns]

# 5) 시대 × 장르별 영화 편수
genre_by_period = df.groupby("five_year_bin")[genre_cols].sum()

print("=== 시대 × 장르별 영화 편수 ===")
print(genre_by_period)




In [ ]:

# ===== 2) 연도 & 5년 구간
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[df["year"] >= 1980].copy()

bins   = list(range(1980, 2025, 5))
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["five_year_bin"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)

# ===== 3) 장르 더미 컬럼 자동 선택
genre_cols = df.loc[:, "Action":"Western"].columns.tolist()

# ===== 4) 시대 × 장르별 편수
count_tbl = df.groupby("five_year_bin")[genre_cols].sum().reindex(labels).fillna(0).astype(int)

# ===== 5) 각 장르 최초 등장 구간
first_seen = {}
for g in genre_cols:
    nz = np.where(count_tbl[g].values > 0)[0]
    first_seen[g] = labels[nz[0]] if len(nz) else None

# ===== 6) 히트맵 시각화
plt.figure(figsize=(14,8))
sns.heatmap(count_tbl.T, cmap="YlGnBu", linewidths=.5, linecolor="lightgray", cbar_kws={"label":"편수"})
plt.title("시대(5년) × 장르별 영화 편수")
plt.xlabel("5년 구간")
plt.ylabel("장르")

# ===== 7) 첫 등장 시점에 별표 표시
for gi, g in enumerate(count_tbl.columns):  # five_year_bin 순회
    pass  # (이건 X, 잘못됨)

for gi, g in enumerate(count_tbl.columns):  # 여기서 g는 구간, 잘못
    pass

# 고침: index=구간, columns=장르 → 전치해서 M 사용
M = count_tbl.T
for gi, g in enumerate(M.index):
    nz = np.where(M.loc[g].values > 0)[0]
    if len(nz) > 0:
        fi = nz[0]
        plt.scatter(fi+0.5, gi+0.5, marker="*", s=120, color="red")  # 첫 등장 별표

plt.tight_layout()
plt.show()

# ===== 8) 장르별 최초 등장 구간 출력
print("=== 장르별 최초 등장 구간 ===")
for g, f in first_seen.items():
    print(f"{g:15s}: {f}")


In [ ]:
plt.figure(figsize=(12,7))
for col in count_tbl.columns:   # 모든 장르
    plt.plot(count_tbl.index, count_tbl[col], marker="o", label=col, alpha=0.7)

plt.title("시대별 모든 장르 영화 편수 추세")
plt.xlabel("5년 구간")
plt.ylabel("편수")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")  # 범례를 밖으로
plt.tight_layout()
plt.show()


파묘시작~

애니메이션-안정성의 원천이 공급 파이프라인(연 1~2편 대형 프랜차이즈) 때문인지,
아니면 소수 스튜디오(디즈니/픽사/드림웍스 등) 스튜디오 효과인지.

Family 동시 태깅의 비중: 실은 “애니=패밀리”가 많아 Family 하락을 일부 상쇄했을 가능성.

예산·표본 규모가 꾸준했는지(대형작 몇 편에 성과가 집중되는지).

In [ ]:

# 애니 편수/성과 점유율
ani_cnt  = df.groupby("period")["Animation"].sum()
ani_perf = df.loc[df["Animation"]==1].groupby("period")["y_result"].sum()
share_ani_cnt  = (ani_cnt / tot * 100).reindex(labels)
share_ani_perf = (ani_perf / df.groupby("period")["y_result"].sum() * 100).reindex(labels)

print("Animation 편수 점유율(%):\n", share_ani_cnt.round(1))
print("Animation 성과 점유율(%):\n", share_ani_perf.round(1))




In [ ]:


def summarize_animation(df, start=1990, end=2025, step=5):
    df = df.copy()
    # 날짜 → 연도 → 5년 구간(period)
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
    df["year"] = df["release_date"].dt.year
    df = df[(df["year"] >= start) & (df["year"] <= end)].copy()

    bins   = list(range(start, end + step, step))
    labels = [f"{y}–{y+step-1}" for y in bins[:-1]]
    df["period"] = pd.cut(df["year"], bins=bins, labels=labels,
                          include_lowest=True, right=True)

    # 전체 분모 (편수/성과 합)
    totals_cnt  = df.groupby("period", observed=False).size().reindex(labels).astype(float)
    totals_perf = df.groupby("period", observed=False)["y_result"].sum().reindex(labels).astype(float)

    # 애니 편수/성과/평균
    ani_cnt  = df.groupby("period", observed=False)["Animation"].sum().reindex(labels).fillna(0.0)
    ani_perf = df.loc[df["Animation"]==1].groupby("period", observed=False)["y_result"] \
                 .sum().reindex(labels).fillna(0.0)
    ani_mean = df.loc[df["Animation"]==1].groupby("period", observed=False)["y_result"] \
                 .mean().reindex(labels)

    # 점유율(%) — 분모=0 또는 편수=0이면 0으로 표기
    share_cnt  = ((ani_cnt  / totals_cnt.replace(0, np.nan))  * 100).fillna(0.0)
    share_perf = ((ani_perf / totals_perf.replace(0, np.nan)) * 100).fillna(0.0)

    excess_pp = (share_perf - share_cnt).fillna(0.0)

    summary = pd.DataFrame({
        "count": ani_cnt,
        "count_share_%": share_cnt,
        "perf_sum": ani_perf,
        "perf_share_%": share_perf,
        "mean_y_result": ani_mean,      # 표본 없으면 NaN 유지
        "excess_perf_pp": excess_pp
    }).reindex(labels).round(2)

    return summary, labels

# ---- 실행 ----
anim_summary, labels = summarize_animation(df, start=1990, end=2025, step=5)
print(anim_summary)







In [ ]:
# (1) 애니 제작 편수
plt.figure(figsize=(9,4))
plt.bar(anim_summary.index.astype(str), anim_summary["count"])
plt.title("Animation 제작 편수 (5년 단위)")
plt.xlabel("Period"); plt.ylabel("편수")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()



In [ ]:

# (2) 점유율(편수% vs 성과%) 오버레이
plt.figure(figsize=(9,4))
plt.plot(anim_summary.index.astype(str), anim_summary["count_share_%"], marker="o", label="편수 점유율(%)")
plt.plot(anim_summary.index.astype(str), anim_summary["perf_share_%"],  marker="o", label="성과 점유율(%)")
plt.title("Animation 점유율(%) — 편수 vs 성과")
plt.xlabel("Period"); plt.ylabel("%")
plt.legend()
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()



In [ ]:


# (3) 성과 합(y_result)
plt.figure(figsize=(9,4))
plt.bar(anim_summary.index.astype(str), anim_summary["perf_sum"])
plt.title("Animation 성과 합 (y_result)")
plt.xlabel("Period"); plt.ylabel("합계 y_result")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()




In [ ]:


# (4) 성과 초과(퍼센트p) = 성과% − 편수%
plt.figure(figsize=(9,4))
plt.plot(anim_summary.index.astype(str), anim_summary["excess_perf_pp"], marker="o")
plt.axhline(0, color="gray", linestyle="--", linewidth=1)
plt.title("Animation 성과 초과 (퍼센트p)")
plt.xlabel("Period"); plt.ylabel("퍼센트p")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()


In [ ]:
xs = np.arange(len(anim_summary))
y  = anim_summary["mean_y_result"].values
n  = anim_summary["count"].values  # 표본수

plt.figure(figsize=(9,4))
plt.plot(xs, y, marker="o")
# 각 점 위에 표본수 표시 (NaN은 건너뜀)
for xi, yi, ni in zip(xs, y, n):
    if not np.isnan(yi):
        plt.text(xi, yi, f"n={int(ni)}", ha="center", va="bottom", fontsize=9)

plt.title("Animation 평균 y_result (5년 단위)")
plt.xlabel("Period"); plt.ylabel("평균 y_result")
plt.xticks(xs, anim_summary.index.astype(str), rotation=45, ha="right")
plt.tight_layout(); plt.show()

In [ ]:

df = df.copy()
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()

bins   = list(range(1990, 2030, 5))
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["period"] = pd.cut(df["year"], bins=bins, labels=labels, include_lowest=True, right=True)

# ==========================================================
# (A) 애니메이션 중 Family 동시태깅 비율 (Animation → Family)
# ==========================================================
ani = df[df["Animation"] == 1].copy()
grp = ani.groupby("period", observed=False).agg(
    anim_n = ("Animation", "size"),
    fam_n  = ("Family", "sum")
).reindex(labels).fillna(0)

rate = (grp["fam_n"] / grp["anim_n"].replace(0, np.nan) * 100).fillna(0)

plt.figure(figsize=(10,4))
plt.plot(rate.index.astype(str), rate.values, marker="o")
# n(애니 편수) 라벨
for x, y, n in zip(rate.index.astype(str), rate.values, grp["anim_n"].astype(int).values):
    if n > 0:
        plt.text(x, y, f"{y:.0f}% (n={n})", ha="center", va="bottom", fontsize=9)
    else:
        plt.text(x, y, "0% (n=0)", ha="center", va="bottom", fontsize=9)
plt.ylim(0, 100)
plt.title("애니메이션 중 Family 동시태깅 비율 (5년 단위)")
plt.xlabel("Period"); plt.ylabel("비율(%)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# ==================================================================
# (B) Family 내부에서 애니 vs 실사 구성비 (Family → Animation / Live)
#     → “패밀리=애니”로 재배치 여부를 100% 스택으로 확인
# ==================================================================
fam = df[df["Family"] == 1].copy()
fam["is_animation"] = fam["Animation"].eq(1).astype(int)
fam["is_live"] = (~fam["Animation"].eq(1)).astype(int)

mix = fam.groupby("period", observed=False)[["is_animation","is_live"]].sum().reindex(labels).fillna(0)
mix_pct = mix.div(mix.sum(axis=1).replace(0, np.nan), axis=0).fillna(0) * 100

ax = mix_pct.rename(columns={"is_animation":"Animation", "is_live":"Live-action"}).plot(
    kind="bar", stacked=True, figsize=(10,4), alpha=0.9
)
ax.set_title("Family 내부 구성비(%) — Animation vs Live-action")
ax.set_xlabel("Period"); ax.set_ylabel("비율(%)")
for container in ax.containers:  # 막대 내부 퍼센트 라벨(작은 값은 생략)
    labels_ = [f"{h.get_height():.0f}%" if h.get_height() >= 8 else "" for h in container]
    ax.bar_label(container, labels=labels_, label_type="center", fontsize=9)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:


# 1) 애니 → 패밀리 (P(Family | Animation))
ani = df[df["Animation"]==1].copy()
p_family_given_ani = (
    ani.groupby("period")["Family"].mean() * 100
).reindex(labels).fillna(0).round(1)

# 2) 패밀리 → 애니 (P(Animation | Family))
fam = df[df["Family"]==1].copy()
p_ani_given_family = (
    fam.groupby("period")["Animation"].mean() * 100
).reindex(labels).fillna(0).round(1)

# 3) 동시태깅 ‘편수’도 같이
both_cnt = df.groupby("period").apply(lambda x: ((x["Animation"]==1)&(x["Family"]==1)).sum()) \
             .reindex(labels).fillna(0).astype(int)

print("P(Family | Animation) %:\n", p_family_given_ani)
print("\nP(Animation | Family) %:\n", p_ani_given_family)
print("\n동시태깅 편수(애니∩패밀리):\n", both_cnt)


In [ ]:

x = np.arange(len(labels))
x_labels = [str(s) for s in labels]

plt.figure(figsize=(10,4))
plt.plot(x, p_family_given_ani.values, marker="o", label="P(Family | Animation)")
plt.plot(x, p_ani_given_family.values, marker="o", label="P(Animation | Family)")
plt.ylim(0, 100)
plt.title("애니↔패밀리 동시태깅 조건부 비율 (5년 단위)")
plt.xlabel("Period"); plt.ylabel("비율(%)")
plt.xticks(x, x_labels, rotation=45, ha="right")
plt.legend()





In [ ]:

# ---------- (2) 동시태깅 편수 + (애니 기준) 동시태깅률 ----------
# 애니 중 동시태깅률 = both_cnt / ani_n * 100
both_rate_in_ani = np.where(ani_n.values>0, (both_cnt.values / ani_n.values) * 100, np.nan)

fig, ax1 = plt.subplots(figsize=(10,4))
ax1.bar(x, both_cnt.values)
ax1.set_title("애니∩패밀리 동시태깅: 편수 & 애니 기준 동시태깅률")
ax1.set_xlabel("Period"); ax1.set_ylabel("동시태깅 편수")
ax1.set_xticks(x); ax1.set_xticklabels(x_labels, rotation=45, ha="right")

# 보조축: 동시태깅률(애니 기준)
ax2 = ax1.twinx()
ax2.plot(x, both_rate_in_ani, marker="o")
ax2.set_ylabel("애니 중 동시태깅률(%)")
ax2.set_ylim(0, 100)

# 막대 위에 편수 라벨
for xi, v in zip(x, both_cnt.values):
    if v > 0:
        ax1.text(xi, v, str(int(v)), ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# (선택) CSV 저장
# anim_summary.to_csv("animation_5y_summary.csv", encoding="utf-8-sig")
